# Recogida de basuras por RL

Código principal para generar la simulación, añadir la red neuronal y el método de entrenamiento.

In [1]:
import math
import os
import sys
import time
import random

import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
import torch
import networkx as nx 
import osmnx as ox

Carga del grafo

In [2]:
# Obtenemos el grafo con los nodos de los contenedores de basura ya incluidos

G = ox.load_graphml("contenedores_resto_FCC_valencia_filtrado_grafo.graphml", node_dtypes={"osmid": str})
nodos, aristas = ox.graph_to_gdfs(G)

# Limitamos el rango de recogida de basuras al barrio de Benimàmet, València

nodos_zona = nodos[(-0.411672 > nodos["x"]) & (nodos["x"] > -0.431842) & (39.506424 > nodos["y"]) & (nodos["y"] > 39.495904)]
nodos_zona.to_csv("nodos_zona.csv", index=True, encoding="utf-8")

nodos_id = set(nodos_zona.index)
aristas_zona = aristas[aristas.index.get_level_values("u").isin(nodos_id) & aristas.index.get_level_values("v").isin(nodos_id)].copy()
aristas_zona.to_csv("aristas_zona.csv", index=True, encoding="utf-8")



Preparación tuplas de nodos_zona y aristas_zona del entorno

In [3]:
# Se trabaja con diccionarios

def generacion_dicc_nodos(nodos_zona):
    nodos_dicc = {}
    for indice, fila in nodos_zona.iterrows():
        nodos_dicc[indice] = {
            "indice" : indice,
            "contenedor" : 1 if fila["contenedor"] == "True" else 0,  # 1 sí,  0 no
            "capacidad_contenedor" : 2 if fila["contenedor"] == "True" else 0, # 1 unidad de llenado sí, 0 unidades de llenado no
            "llenado" :  0.5 if fila["contenedor"] == "True" else 0,  # 0.5 (50%) (valor inicial aleatorio) sí,  0 (0%) no
            "posicion_camion" : 0,                                    # 1 sí,  0 no
            "llenado_camion" : 0                                      # Dato compartido en todos los nodos, normalizado a 1   
        }
    
    return nodos_dicc

def generacion_dicc_aristas(aristas_zona):
    aristas_dicc = {}
    for indice, fila in aristas_zona.iterrows():
        aristas_dicc[indice] = {
            "indice" : fila["osmid"], 
            "desde" : indice[0],
            "hasta" : indice[1],
            "distancia" : fila["length"],  # m
            "tiempo_recorrido" : fila["travel_time"], # s
            "velocidad_media" : fila["speed_kph"], # Km/h (velocidad indicada por graphml)
            "velocidad_max" : 30, # Km/h (Consideración inicial vias interubanas) 
            "via" : fila["highway"]
        }

    return aristas_dicc


nodos_dicc = generacion_dicc_nodos(nodos_zona)
aristas_dicc = generacion_dicc_aristas(aristas_zona)


Modificación indices de los nodos para trabajar despues con las GNN (se almacena la relación para mostrar resultados)

In [4]:
def modificacion_indices(nodos_dicc, aristas_dicc):
    
    nodos_dicc_str = {str(k): v for k, v in nodos_dicc.items()}

    mapeo = {osmid: idx for idx, osmid in enumerate(nodos_dicc_str.keys())}
    mapeo_inverso = {idx: osmid for osmid, idx in mapeo.items()}

    nodos_dicc_ind = {}
    for osmid, data in nodos_dicc_str.items():
        new_idx = mapeo[osmid]
        nodo_copy = data.copy()
        nodo_copy["indice"] = new_idx
        nodos_dicc_ind[new_idx] = nodo_copy

    aristas_dicc_ind = {}
    for osmid, data in aristas_dicc.items():
        u_idx = mapeo[str(data["desde"])]
        v_idx = mapeo[str(data["hasta"])]
        arista_copy = data.copy()
        arista_copy["desde"] = u_idx
        arista_copy["hasta"] = v_idx
        aristas_dicc_ind[osmid] = arista_copy

    return nodos_dicc_ind, aristas_dicc_ind, mapeo, mapeo_inverso

nodos_indice, aristas_indice, mapeo, mapeo_inverso = modificacion_indices(nodos_dicc, aristas_dicc)
print(mapeo_inverso[113])
print(nodos_indice)
print(aristas_indice)

344427875
{0: {'indice': 0, 'contenedor': 0, 'capacidad_contenedor': 0, 'llenado': 0, 'posicion_camion': 0, 'llenado_camion': 0}, 1: {'indice': 1, 'contenedor': 0, 'capacidad_contenedor': 0, 'llenado': 0, 'posicion_camion': 0, 'llenado_camion': 0}, 2: {'indice': 2, 'contenedor': 0, 'capacidad_contenedor': 0, 'llenado': 0, 'posicion_camion': 0, 'llenado_camion': 0}, 3: {'indice': 3, 'contenedor': 0, 'capacidad_contenedor': 0, 'llenado': 0, 'posicion_camion': 0, 'llenado_camion': 0}, 4: {'indice': 4, 'contenedor': 0, 'capacidad_contenedor': 0, 'llenado': 0, 'posicion_camion': 0, 'llenado_camion': 0}, 5: {'indice': 5, 'contenedor': 0, 'capacidad_contenedor': 0, 'llenado': 0, 'posicion_camion': 0, 'llenado_camion': 0}, 6: {'indice': 6, 'contenedor': 0, 'capacidad_contenedor': 0, 'llenado': 0, 'posicion_camion': 0, 'llenado_camion': 0}, 7: {'indice': 7, 'contenedor': 0, 'capacidad_contenedor': 0, 'llenado': 0, 'posicion_camion': 0, 'llenado_camion': 0}, 8: {'indice': 8, 'contenedor': 0, 'ca

Preparación y entremaniento agente

In [5]:
from env_basuras_final import RecogidaBasurasEnv

# Asegúrate de tener estos dicts ya definidos en la sesión:
# nodos_indice, aristas_indice

env = RecogidaBasurasEnv(nodos_indice, aristas_indice, mascara=True, seed=22)



Función evaluación modelo

In [6]:
def eval_episode(env, model, deterministic=True, max_steps=500):
    obs, _ = env.reset()
    ep_rew, steps = 0.0, 0

    while steps < max_steps:
        act, _ = model.predict(obs, deterministic=deterministic)
        act = np.array(act).squeeze()
        if act.ndim > 1:
            act = act[0]
        tipo, destino = int(act[0]), int(act[1])

        obs, r, term, trunc, _ = env.step([tipo, destino])
        ep_rew += float(r)
        steps += 1
        if term or trunc:
            break

    print(f"[EVAL] determinista={deterministic} | recompensa={ep_rew:.2f} | steps={steps}")

# Entrenamiento A2C con paralelización

Ronda 6

In [7]:
# main.py
# ------------------------------------------------------------
# Lanza el entrenamiento con multiprocessing y guardado a CARPETAS.
# Asegúrate de definir/importar nodos_indice y aristas_indice ANTES.
# ------------------------------------------------------------
import os
from A2C_algoritmo import train_a2c

# Aquí debes tener ya:
# from tu_modulo_grafo import nodos_indice, aristas_indice
# o haberlos construido arriba en este mismo archivo/notebook.

def main():
    cfg = dict(
        total_timesteps = 10_000_000,
        n_envs = 6,
        n_steps = 32,            
        learning_rate = 4e-4,
        ent_coef_inicial = 1.0,
        ent_coef_final = 0.04,
        gamma = 0.98,            # Reward
        beta_int = 0.2,         # Intrínseco
        alpha = 0.99,            # Intrínseco
        seed = 22,
        device = "cuda",
        run_name = "ronda_6",
        models_dir = "./models/a2c",
        tb_dir = "./logs/tb-a2c",
        save_freq = 500_000,     # checkpoints cada 450k a carpeta
    )

    model, final_dir = train_a2c(
        nodos_indice = nodos_indice,
        aristas_indice = aristas_indice,
        **cfg
    )

    print("[FIN] Entrenamiento completado.")
    print("Carpeta final:", final_dir)

    return model


if __name__ == "__main__":
    model_6 = main()

c:\Users\jgilg\Desktop\Master_IA_VIU\TFM\codigo_github\rl_env\lib\site-packages\torch_geometric\typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: Could not find module 'C:\Users\jgilg\Desktop\Master_IA_VIU\TFM\codigo_github\rl_env\Lib\site-packages\libpyg.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
c:\Users\jgilg\Desktop\Master_IA_VIU\TFM\codigo_github\rl_env\lib\site-packages\torch_geometric\typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: [WinError 127] No se encontró el proceso especificado
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
c:\Users\jgilg\Desktop\Master_IA_VIU\TFM\codigo_github\rl_env\lib\site-packages\torch_geometric\typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: [

[INFO] VecEnv usado: SubprocVecEnv | n_envs=6
Using cuda device
Logging to ./logs/tb-a2c\ronda_6\csv


c:\Users\jgilg\Desktop\Master_IA_VIU\TFM\codigo_github\rl_env\lib\site-packages\stable_baselines3\common\vec_env\vec_monitor.py:44: UserWarning: The environment is already wrapped with a `Monitor` wrapperbut you are wrapping it with a `VecMonitor` wrapper, the `Monitor` statistics will beoverwritten by the `VecMonitor` ones.
  warnings.warn(


------------------------------------
| rollout/              |          |
|    ep_len_mean        | 31.3     |
|    ep_rew_mean        | 2.9      |
| time/                 |          |
|    fps                | 422      |
|    iterations         | 100      |
|    time_elapsed       | 45       |
|    total_timesteps    | 19200    |
| train/                |          |
|    ent_coef           | 0.998    |
|    entropy_loss       | -0.182   |
|    explained_variance | 0.00126  |
|    learning_rate      | 0.0004   |
|    n_updates          | 99       |
|    policy_loss        | -0.0147  |
|    value_loss         | 1.42     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 42.6     |
|    ep_rew_mean        | 3.66     |
| time/                 |          |
|    fps                | 429      |
|    iterations         | 200      |
|    time_elapsed       | 89       |
|    total_timesteps    | 38400    |
|

In [11]:
from stable_baselines3.common.monitor import Monitor

eval_env = Monitor(
    RecogidaBasurasEnv(
        nodos_indice=nodos_indice,
        aristas_indice=aristas_indice,
        steps_maximo=1200,
        mascara=True,
        seed=123,
    )
)

for i in range(10):
    eval_episode(eval_env, model_6, deterministic=True, max_steps=1200)

[EVAL] determinista=True | recompensa=4.12 | steps=15
[EVAL] determinista=True | recompensa=-0.05 | steps=8
[EVAL] determinista=True | recompensa=4.12 | steps=15
[EVAL] determinista=True | recompensa=-0.05 | steps=8
[EVAL] determinista=True | recompensa=4.12 | steps=15
[EVAL] determinista=True | recompensa=-0.05 | steps=8
[EVAL] determinista=True | recompensa=4.12 | steps=15
[EVAL] determinista=True | recompensa=-0.05 | steps=8
[EVAL] determinista=True | recompensa=4.12 | steps=15
[EVAL] determinista=True | recompensa=-0.05 | steps=8


## Extra

Extracción pesos del modelo ya entrenado y guardado en carpeta

In [9]:
import torch
from stable_baselines3 import A2C
from sb3_policy_mascara import A2CPolicyGNNMasked

# recrea el vec_env con los mismos spaces:
# (usa DummyVecEnv si solo vas a predecir/evaluar)
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor
from env_basuras_final import RecogidaBasurasEnv

def make_eval_env():
    return Monitor(RecogidaBasurasEnv(nodos_indice, aristas_indice, steps_maximo=1200, mascara=True, seed=123))

vec_env = DummyVecEnv([make_eval_env])

# crea el modelo "vacío" (mismas policy_kwargs que en entrenamiento)
policy_kwargs = dict(hidden_dim=128, in_node_features=5, in_edge_features=2, n_tipos=2, max_nodes=len(nodos_indice), gnn_layers=3)
model = A2C(policy=A2CPolicyGNNMasked, env=vec_env, policy_kwargs=policy_kwargs, device="cuda", verbose=0)

# carga pesos desde carpeta
folder = "./models/a2c/ronda_4/final"    # Varia dependiendo run
state_path = os.path.join(folder, "policy_state.pt")
opt_path   = os.path.join(folder, "optimizer_state.pt")

sd = torch.load(state_path, map_location="cuda")
model.policy.load_state_dict(sd)

if os.path.exists(opt_path):
    opt_sd = torch.load(opt_path, map_location="cuda")
    if hasattr(model.policy, "optimizer") and model.policy.optimizer is not None:
        model.policy.optimizer.load_state_dict(opt_sd)

print("Pesos cargados desde carpeta:", folder)

Pesos cargados desde carpeta: ./models/a2c/ronda_4/final


In [10]:
eval_episode(env, model, deterministic=True, max_steps=1200)

[EVAL] determinista=True | recompensa=4.12 | steps=15
